In [7]:
import datetime
import json
import pandas as pd

In [8]:
def create_segment(trip):
    srcx, srcy = trip[['Xsource', 'Ysource']]
    tgtx, tgty = trip[['Xtarget', 'Ytarget']]
    
    epoch = datetime.datetime.utcfromtimestamp(0)
    
    dt = pd.to_timedelta(trip['Departtime']) + epoch
    tt = pd.to_timedelta(trip['Traveltime'], unit='s')
    
    segments = [[srcx, srcy, dt.timestamp()], [tgtx, tgty, (dt+tt).timestamp()]]
    return segments


In [9]:
descr = pd.read_csv(r'trs_mdl_descr.csv', index_col=[0, 1])

In [10]:
descr.head(2)

Source     Target     Direction Timeperiod Purpose  \
Tourid Legseqidx                                                          
1      0          E01014387  E01014370      FromHome         AM     HBO   
       1          E01014370  E01014379  NonHomeBased         PM    NHBO   

                  Odpairid  Distance  Traveltime Departtime   Xsource  \
Tourid Legseqidx                                                        
1      0                51     470.9       470.9   08:49:07 -2.358667   
       1                 1     951.8       951.8   18:05:30 -2.359580   

                    Ysource   Xtarget    Ytarget  
Tourid Legseqidx                                  
1      0          51.359112 -2.359580  51.383284  
       1          51.383284 -2.426716  51.347451

In [11]:
segment = descr.apply(create_segment, axis=1)
segment = segment.groupby(level=0).sum() # extends lists

In [12]:
data = descr.groupby(level=0).agg(list)
data['Segment'] = segment
data = data.add_suffix('s')

In [13]:
data.head()

,Sources,Targets,Directions,Timeperiods,Purposes,Odpairids,Distances,Traveltimes,Departtimes,Xsources,Ysources,Xtargets,Ytargets,Segments
Tourid,,,,,,,,,,,,,,
1,"[E01014387, E01014370, E01014379]","[E01014370, E01014379, E01014387]","[FromHome, NonHomeBased, ToHome]","[AM, PM, PM]","[HBO, NHBO, HBO]","[51, 1, 22]","[470.9, 951.8, 606.1]","[470.9, 951.8, 606.1]","[08:49:07, 18:05:30, 18:41:35]","[-2.358666776, -2.35958041, -2.426716202]","[51.35911178, 51.38328426, 51.3474509]","[-2.35958041, -2.426716202, -2.358666776]","[51.38328426, 51.3474509, 51.35911178]","[[-2.358666776, 51.35911178, 31747.0], [-2.359..."
2,"[E01014387, E01033342]","[E01033342, E01014387]","[FromHome, ToHome]","[AM, IP3]","[HBO, HBO]","[56, 3814]","[1814.0, 1854.4]","[1814.0, 1854.4]","[08:55:49, 19:01:04]","[-2.358666776, -2.590504446]","[51.35911178, 51.4591305]","[-2.590504446, -2.358666776]","[51.4591305, 51.35911178]","[[-2.358666776, 51.35911178, 32149.0], [-2.590..."
3,"[E01014387, E01014540]","[E01014540, E01014387]","[FromHome, ToHome]","[IP3, OP2]","[HBO, HBO]","[52, 808]","[1807.9, 1809.6]","[1807.9, 1809.6]","[20:09:08, 22:24:38]","[-2.358666776, -2.592629778]","[51.35911178, 51.45234647]","[-2.592629778, -2.358666776]","[51.45234647, 51.35911178]","[[-2.358666776, 51.35911178, 72548.0], [-2.592..."
4,"[E01014396, E01033345]","[E01033345, E01014396]","[FromHome, ToHome]","[OP1, IP1]","[HBW, HBW]","[91, 3864]","[1114.8, 993.9]","[1114.8, 993.9]","[05:14:30, 12:30:14]","[-2.480693215, -2.600578545]","[51.41533884, 51.45036181]","[-2.600578545, -2.480693215]","[51.45036181, 51.41533884]","[[-2.480693215, 51.41533884, 18870.0], [-2.600..."
5,"[E01014396, E01014379, E01014407]","[E01014379, E01014407, E01014396]","[FromHome, NonHomeBased, ToHome]","[AM, IP1, IP2]","[HBW, NHBW, HBO]","[78, 24, 139]","[1247.0, 849.9, 823.0]","[1247.0, 849.9, 823.0]","[09:22:13, 12:05:21, 14:37:24]","[-2.480693215, -2.426716202, -2.365562852]","[51.41533884, 51.3474509, 51.38179054]","[-2.426716202, -2.365562852, -2.480693215]","[51.3474509, 51.38179054, 51.41533884]","[[-2.480693215, 51.41533884, 33733.0], [-2.426..."


In [19]:
d = data.reset_index().to_dict('records')

In [20]:
#d = data.to_dict(orient='records')

In [21]:
# Get a file object with write permission.
f = open('data.json', 'w')
# Save dict data into the JSON file.
json.dump(d, f)